In [1]:
%pip install kagglehub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rtatman/deceptive-opinion-spam-corpus")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\pokes\.cache\kagglehub\datasets\rtatman\deceptive-opinion-spam-corpus\versions\2


In [1]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold

In [4]:
df = pd.read_csv(path + "\\deceptive-opinion.csv")

In [5]:
df

,deceptive,hotel,polarity,source,text
0,truthful,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...
1,truthful,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...
2,truthful,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...
3,truthful,omni,positive,TripAdvisor,The Omni Chicago really delivers on all fronts...
4,truthful,hyatt,positive,TripAdvisor,I asked for a high floor away from the elevato...
...,...,...,...,...,...
1595,deceptive,intercontinental,negative,MTurk,Problems started when I booked the InterContin...
1596,deceptive,amalfi,negative,MTurk,The Amalfi Hotel has a beautiful website and i...
1597,deceptive,intercontinental,negative,MTurk,The Intercontinental Chicago Magnificent Mile ...
1598,deceptive,palmer,negative,MTurk,"The Palmer House Hilton, while it looks good i..."


In [6]:
X_text = df["text"].values
y = df["deceptive"].apply(lambda x: 1 if x == "deceptive" else 0).values

In [24]:
from sklearn.decomposition import PCA
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics import precision_score, recall_score, f1_score

In [30]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

n_clusters = 32

accs, precisions, recalls, f1s = [], [], [], []
for train_index, test_index in kf.split(X_text):
  X_train, X_test = X_text[train_index], X_text[test_index]
  y_train, y_test = y[train_index], y[test_index]
  vectorizer = TfidfVectorizer()
  X_train_tfidf = vectorizer.fit_transform(X_train)
  pca = PCA(n_components=20)
  X_train_tfidf = pca.fit_transform(X_train_tfidf.toarray())
  X_test_tfidf = vectorizer.transform(X_test)
  X_test_tfidf = pca.transform(X_test_tfidf.toarray())
  kmeans = KMeans(n_clusters=n_clusters, random_state=42)
  clusters = kmeans.fit_transform(X_train_tfidf)
  cluster_to_label = {}
  for i in range(n_clusters):
    cluster_indices = np.where(kmeans.labels_ == i)[0]
    cluster_labels = y_train[cluster_indices]
    if len(cluster_labels) > 0:
      cluster_to_label[i] = np.bincount(cluster_labels).argmax()
    else:
      cluster_to_label[i] = 0  # Default label if no samples in cluster
  pred_clusters = kmeans.predict(X_test_tfidf)
  y_pred = np.array([cluster_to_label[cluster] for cluster in pred_clusters])
  accuracy = np.mean(y_pred == y_test)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)
  accs.append(accuracy)
  precisions.append(precision)
  recalls.append(recall)
  f1s.append(f1)
print("Average Accuracy:", np.mean(accs))
print("Average Precision:", np.mean(precisions))
print("Average Recall:", np.mean(recalls))
print("Average F1 Score:", np.mean(f1s))


Average Accuracy: 0.7125
Average Precision: 0.7007448574644083
Average Recall: 0.7386710659528634
Average F1 Score: 0.7185411167521144
